In [1]:
# host addresses
SERVER0 = ""  
SERVER1 = ""

In [2]:
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd
from processor_grameval2020 import ProcessorGramEval2020


address_udpipe = (SERVER0, 3344)
address_qbic = (SERVER1, 3334)

ppl_udpipe = PipelineCommon([
    (ProcessorRemote(address_udpipe[0], address_udpipe[1], '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens',
      'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
     ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'})
])

ppl_qbic = PipelineCommon([
    (ProcessorRemote(address_udpipe[0], address_udpipe[1], '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens'}),
    (ProcessorRemote(address_qbic[0], address_qbic[1], '0'),
     ['tokens', 'sentences'],
     {'lemma': 'lemma',
      'postag': 'postag',
      'morph': 'morph',
      'syntax_dep_tree': 'syntax_dep_tree'})
])

In [3]:
text = "По нашим данным, кости обнаружили при рытье котлована торгового центра «Европа» еще в 2006 году."

#### UDPipe output:

In [4]:
from print_syntax import print_syntax_dep_tree

result = ppl_udpipe(text)
print_syntax_dep_tree(result)

      ┌──► По         case
      │ ┌► нашим      det
    ┌►└─└─ данным     parataxis
    │ └──► ,          punct
    │   ┌► кости      nsubj
┌─┌─└─┌─└─ обнаружили 
│ │   │ ┌► при        case
│ │ ┌─└►└─ рытье      obl
│ │ │ └──► котлована  nmod
│ │ │   ┌► торгового  amod
│ │ └►┌─└─ центра     nmod
│ │   │ ┌► «          punct
│ │   └►└─ Европа     appos
│ │   └──► »          punct
│ │ ┌────► еще        obl
│ │ │ ┌──► в          case
│ │ │ │ ┌► 2006       nummod
│ └►└─└─└─ году       obl
└────────► .          punct


#### qbic output: 

In [5]:
result = ppl_qbic(text)
print_syntax_dep_tree(result)

        ┌──► По         case
        │ ┌► нашим      det
      ┌►└─└─ данным     parataxis
      │ └──► ,          punct
      │   ┌► кости      obj
┌─┌───└─┌─└─ обнаружили 
│ │     │ ┌► при        case
│ │     └►└─ рытье      obl
│ │   ┌─└──► котлована  nmod
│ │   │   ┌► торгового  amod
│ │ ┌─└──►└─ центра     nmod
│ │ │     ┌► «          punct
│ │ └──►┌─└─ Европа     appos
│ │     └──► »          punct
│ │   ┌────► еще        advmod
│ │   │ ┌──► в          case
│ │   │ │ ┌► 2006       amod
│ └──►└─└─└─ году       obl
└──────────► .          punct
